## ТИПЫ ДАННЫХ

Работа будет проводиться на той же самой бд, что и ранее.

## ДАТАСЕТ

В данном блоке мы будем работать с данными о компании, организующей ***перевозки*** грузов.

Интересующие данные хранятся в таблицах:

- city (справочник городов)
    * city_id	integer	уникальный идентификатор города, первичный ключ
    * city_name	text	название города
    * state	text	штат, к которому относится город
    * population	integer	население города
    * area	numeric	площадь города
    
- customer (справочник клиентов)
    * cust_id	integer	уникальный идентификатор клиента, первичный ключ
    * cust_name	text	название клиента
    * annual_revenue	numeric	ежегодная выручка
    * cust_type	text	тип пользователя
    * address	text	адрес
    * zip	integer	почтовый индекс
    * phone	text	телефон
    * city_id	integer	идентификатор города, внешний ключ к таблице city

- driver (справочник водителей)
    * driver_id	integer	уникальный идентификатор водителя, первичный ключ
    * first_name	text	имя водителя
    * last_name	text	фамилия водителя
    * address	text	адрес водителя
    * zip_code	integer	почтовый индекс водителя
    * phone	text	телефон водителя
    * city_id	integer	идентификатор города водителя, внешний ключ к таблице city

- shipment (таблица с данными непосредственно о доставках)
    * ship_id	integer	уникальный идентификатор доставки, первичный ключ
    * cust_id	integer	идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer
    * weight	numeric	вес посылки
    * truck_id	integer	идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck
    * driver_id	integer	идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver
    * city_id	integer	идентификатор города в который совершена доставка, внешний ключ к таблице city
    * ship_date	date	дата доставки

- truck (информация о грузовиках, на которых осуществляются перевозки)
    * truck_id	integer	Уникальный идентификатор грузовика, первичный ключ
    * make	text	Производитель грузовика
    * model_year	integer	Дата выпуска грузовика

## ОСНОВНЫЕ ТИПЫ ДАННЫХ В POSTGRESQL

**SQL** — это язык со строгой типизацией, в котором каждый элемент данных имеет некоторый тип, определяющий его поведение и допустимое использование.

Типы данных в PostgreSQL можно разделить на несколько групп. К основным относятся:

* числовые типы — для хранения чисел (целых и дробных);

* типы даты/времени — для хранения даты, времени, часовых поясов;

* символьные типы — для хранения символов или строк;

* логический тип — для хранения значений типа «истина», «ложь».

## О ВАЖНОСТИ ТИПИЗАЦИИ

В зависимости от требований к хранимой информации необходимо правильно применять типы данных. На это есть как минимум две причины.

1. Разные типы данных могут занимать разный объём памяти.

Например, если заранее известно, что поле таблицы будет принимать только небольшие числовые значения от 1 до 10, то можно не задавать для него тип **bigint** с максимально возможными хранимыми значениями, достаточно будет типа **smallint**.

2. На преобразование типов данных тратится время.

В некоторых случаях может потребоваться преобразование одного типа в другой — например, если нужно извлечь число из строки и произвести с ним какие-либо действия. Для подобных преобразований нужно использовать дополнительные функции. Иногда без этого не обойтись, но по возможности лучше хранить данные в полях соответствующего типа.

## TIMESTAMP

Timestamp — наиболее распространённый тип данных, так как он содержит и дату, и время, а также используется в любых логах событий, временных рядах и в большинстве системных таблиц.

Согласно стандарту ISO, значение выглядит как "2019-07-14 01:35:44.702165+00", где перечислены через точку год-месяц-день, время и часовой пояс.

Для получения текущего значения даты и времени в Postgres используются функции CURRENT_TIMESTAMP (есть в стандарте SQL) и NOW() (есть в большинстве баз данных).

``` sql

-- Текущая дата и время
select 
now(), 
current_timestamp

-- Показывает временную зону (GMT например это наиболее частая установка по умолчанию для баз данных)
show timezone

-- Показывает таблицу часовых поясов в системном справочнике pg_timezone_names
select 
* 
from pg_timezone_names

-- Показывает временную зону. 
-- Указание at time zone позволяет переводить дату/время без часового пояса в дату/время с часовым поясом и обратно, а также пересчитывать значения времени для различных часовых поясов.
select 
now() 
at 
time zone 'Europe/Moscow'

```

## TIMESTAMP WITH TIME ZONE

Timestamp with time zone позволяет хранить сведения о часовом поясе, что может быть удобно при анализе географически распределённых временных данных для единообразия хранения.

Предположим, в вашей компании в базу подтягивается время прихода сотрудников на работу. Вы пришли в 10 утра по Москве, а в Екатеринбурге в это время — полдень. Чтобы ваши коллеги из Екатеринбурга поняли, что вы пришли на работу вовремя, им нужно помнить про разницу в часовых поясах. А теперь представим, что пользователи БД разбросаны по всему миру и всем им необходимо помнить о разнице во времени и учитывать её при сверках с другими регионами.

Согласитесь, неудобно. Поэтому временные метки лучше всего хранить в базе в едином часовом поясе.

Системный справочник pg_timezone_names содержит:

* name	text	Название часового пояса

* abbrev	text	Сокращение часового пояса

* utc_offset	interval	Смещение от GMT (положительное — к востоку от Гринвича, отрицательное — к западу от Гринвича)

* is_dst	boolean	Принимает значение True, если текущие данные отражают летнее время


В таблице ниже приведены примеры как работает at time zone:

| timestamp without time zone AT TIME ZONE часовой_пояс	| timestamp with time zone |	- Воспринимает время, заданное без указания часового пояса, как время в указанном часовом поясе.

| timestamp with time zone AT TIME ZONE часовой_пояс |	timestamp without time zone |   - Переводит значение timestamp с часовым поясом в другой часовой пояс, но не сохраняет информацию о нём в результате.

| time with time zone AT TIME ZONE часовой_пояс |	time with time zone | -	Переводит время с часовым поясом в другой часовой пояс.

``` sql

-- Давайте узнаем, сколько сейчас времени в другом регионе, например Лос-Анджелесе. 
-- Напишите запрос, который выведет текущие время и дату в часовом поясе Лос-Анджелеса ("America/Los_Angeles"). 
-- Столбец в выдаче — now (время и дата в нужном часовом поясе).

select
current_timestamp
at time zone 'America/Los_Angeles' as now
```

## DATE

Реализация типа date предельно проста. Cтоит отметить только, что тип timestamp (with/without time zone) можно легко перевести в соответствующую дату, используя синтаксис

``` sql
"timestamp_column"::date
-- И наоборот, тип date преобразуется в timestamp (дата и 00:00:00) с помощью

"date_column"::timestamp
-- Для получения текущей даты можно использовать
select CURRENT_DATE

-- или
select now()::date

-- Используя следующий подзапрос можно вывести дату в ts в Московском часовом поясе и в поясе UTC:
-- Столбцы в выдаче: dt_msk (дата в московском часовом поясе), dt_utc (дата в UTC).

with x as
(
select '2018-12-31 21:00:00+00'::timestamp with time zone ts
)
select
(ts at time zone 'Europe/Moscow')::date as dt_msk,
(ts at time zone 'UTC')::date as dt_utc
from
x
```

## INTERVAL

Interval — тип данных, позволяющий хранить разницу между двумя временными метками. 

Интервалы хранят данные в трёх отдельных полях — месяцах, днях, секундах. Это сделано из-за того, что количество дней в месяце и часов в дне может быть разным. Пример значения такого типа: "195 days -10:52:23.563955".